In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display


# object detection module
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# import config
import config

# PATCHES
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

def load_model(path):
    model = tf.saved_model.load(path)
    model = model.signatures['serving_default']
    return model



def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict


def show_inference(model, image_path):
  # return dict
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)

  return output_dict



# LOADING LABEL MAP
category_index = label_map_util.create_category_index_from_labelmap(config.PATH_TO_LABELS, use_display_name=True)


# DETECTION
detection_model = load_model(config.PATH_MODEL)


# OUTPUT

output = show_inference(detection_model,config.PATH_IMAGE)

{'detection_scores': array([0.9406907 , 0.9345027 , 0.23088235, 0.22518939, 0.17249912,
       0.1396198 , 0.13212034, 0.09975746, 0.08992714, 0.08888808,
       0.08091754, 0.07895234, 0.07818329, 0.07565752, 0.07331279,
       0.0707489 , 0.06597456, 0.06404546, 0.06388   , 0.06381172,
       0.0638018 , 0.06050137, 0.05898514, 0.058983  , 0.05761427,
       0.05641398, 0.05638802, 0.05500007, 0.0549573 , 0.05426419,
       0.05425417, 0.0540958 , 0.0533593 , 0.05279237, 0.05228513,
       0.05219474, 0.05164075, 0.0506601 , 0.05030194, 0.04837742,
       0.04822728, 0.04720658, 0.04673675, 0.04671779, 0.04644245,
       0.04614913, 0.04420188, 0.04286546, 0.04251266, 0.0417434 ,
       0.0416657 , 0.0413734 , 0.04096133, 0.04053533, 0.03990963,
       0.03988713, 0.03984368, 0.03930986, 0.0392389 , 0.0387798 ,
       0.03877512, 0.03853002, 0.03816223, 0.03815907, 0.03742462,
       0.03689882, 0.03677598, 0.03671169, 0.03654769, 0.03635332,
       0.03612375, 0.03610823, 0.03601933

In [3]:
output_dict = show_inference(detection_model,config.PATH_IMAGE)

In [12]:
output_dict['detection_boxes']

array([[3.90840173e-02, 1.92150325e-02, 8.72103572e-01, 3.15773547e-01],
       [1.09515011e-01, 4.02835637e-01, 9.24646080e-01, 9.73047853e-01],
       [5.07123709e-01, 3.85651469e-01, 8.76479745e-01, 7.03940868e-01],
       [6.38730824e-02, 2.81166613e-01, 9.16385174e-01, 8.24097216e-01],
       [2.15276718e-01, 3.96740973e-01, 9.12904263e-01, 6.79073751e-01],
       [5.27002215e-01, 3.72713149e-01, 8.83502722e-01, 6.94296658e-01],
       [0.00000000e+00, 5.25425553e-01, 2.53364921e-01, 7.16124296e-01],
       [7.37803578e-02, 2.98963934e-01, 9.20516431e-01, 9.38026786e-01],
       [5.31044006e-01, 3.84129941e-01, 9.15436625e-01, 8.53949010e-01],
       [1.45327568e-01, 3.87902439e-01, 9.06316280e-01, 9.78849113e-01],
       [2.08420455e-01, 2.78770208e-01, 8.26954901e-01, 6.09079719e-01],
       [3.48854303e-01, 3.05225790e-01, 8.99384737e-01, 7.33766973e-01],
       [7.35026598e-03, 5.13690710e-02, 6.02022588e-01, 8.20265830e-01],
       [7.01456368e-02, 4.42671865e-01, 6.44431710e

In [5]:
output_dict['detection_classes']

array([18, 18, 18, 18, 18, 33,  1, 63, 18, 21,  3, 18,  1, 18, 21, 18, 62,
       18, 18, 62,  1, 62, 18,  3,  1, 21,  1,  1,  1,  1,  1, 18,  1,  1,
        1,  1, 62,  1,  1,  1,  1,  1, 62, 18,  1, 63,  1,  1, 84,  1,  1,
       18,  1,  1, 62, 21, 57, 18, 18, 62, 18,  1,  1,  1,  1,  1,  1,  1,
       18, 62,  1, 88, 47,  1, 47, 62, 84,  1, 18,  1,  1,  1,  1, 84, 18,
       62, 18, 21,  1, 62, 62,  1, 18, 21,  1, 18, 47,  1,  1, 47])

In [6]:
output_dict['detection_scores']

array([0.9406907 , 0.9345027 , 0.23088235, 0.22518939, 0.17249912,
       0.1396198 , 0.13212034, 0.09975746, 0.08992714, 0.08888808,
       0.08091754, 0.07895234, 0.07818329, 0.07565752, 0.07331279,
       0.0707489 , 0.06597456, 0.06404546, 0.06388   , 0.06381172,
       0.0638018 , 0.06050137, 0.05898514, 0.058983  , 0.05761427,
       0.05641398, 0.05638802, 0.05500007, 0.0549573 , 0.05426419,
       0.05425417, 0.0540958 , 0.0533593 , 0.05279237, 0.05228513,
       0.05219474, 0.05164075, 0.0506601 , 0.05030194, 0.04837742,
       0.04822728, 0.04720658, 0.04673675, 0.04671779, 0.04644245,
       0.04614913, 0.04420188, 0.04286546, 0.04251266, 0.0417434 ,
       0.0416657 , 0.0413734 , 0.04096133, 0.04053533, 0.03990963,
       0.03988713, 0.03984368, 0.03930986, 0.0392389 , 0.0387798 ,
       0.03877512, 0.03853002, 0.03816223, 0.03815907, 0.03742462,
       0.03689882, 0.03677598, 0.03671169, 0.03654769, 0.03635332,
       0.03612375, 0.03610823, 0.03601933, 0.0358645 , 0.03582

In [10]:
category_index

{1: {'id': 1, 'name': 'person'},
 2: {'id': 2, 'name': 'bicycle'},
 3: {'id': 3, 'name': 'car'},
 4: {'id': 4, 'name': 'motorcycle'},
 5: {'id': 5, 'name': 'airplane'},
 6: {'id': 6, 'name': 'bus'},
 7: {'id': 7, 'name': 'train'},
 8: {'id': 8, 'name': 'truck'},
 9: {'id': 9, 'name': 'boat'},
 10: {'id': 10, 'name': 'traffic light'},
 11: {'id': 11, 'name': 'fire hydrant'},
 13: {'id': 13, 'name': 'stop sign'},
 14: {'id': 14, 'name': 'parking meter'},
 15: {'id': 15, 'name': 'bench'},
 16: {'id': 16, 'name': 'bird'},
 17: {'id': 17, 'name': 'cat'},
 18: {'id': 18, 'name': 'dog'},
 19: {'id': 19, 'name': 'horse'},
 20: {'id': 20, 'name': 'sheep'},
 21: {'id': 21, 'name': 'cow'},
 22: {'id': 22, 'name': 'elephant'},
 23: {'id': 23, 'name': 'bear'},
 24: {'id': 24, 'name': 'zebra'},
 25: {'id': 25, 'name': 'giraffe'},
 27: {'id': 27, 'name': 'backpack'},
 28: {'id': 28, 'name': 'umbrella'},
 31: {'id': 31, 'name': 'handbag'},
 32: {'id': 32, 'name': 'tie'},
 33: {'id': 33, 'name': 'suitc